In [0]:
from pyspark.sql.functions import date_format,year, row_number, rank, col,max,desc

In [0]:
%sql
 
REFRESH TABLE title_year2;
REFRESH TABLE df;
REFRESH TABLE title_year;
REFRESH TABLE new_corona_set

##Zanimljivo je da se u oktobru znacajno povecava broj izdatih igara

In [0]:
title_year = sqlContext.sql("select * from `title_year` ORDER BY year DESC")
date_game_count = title_year.groupBy("date").count()
display(date_game_count)

date,count
11 2019,77
8 2021,86
8 2020,110
3 2018,109
3 2017,99
5 2020,87
5 2018,126
9 2020,97
4 2020,73
1 2017,52


##Poredjene broja igara za platforme po mesecima u 2021 godini

In [0]:
table = sqlContext.sql("select * from `title_year2` ")

platform_date = table.groupBy("date").pivot("platform",["PC",   "PlayStation_4", "PlayStation_5", "Xbox_One"]).count()
display(platform_date)

date,PC,PlayStation_4,PlayStation_5,Xbox_One
8 2021,30,4,14,3
4 2021,25,11,8,2
3 2021,30,12,12,5
7 2021,31,10,7,3
6 2021,30,12,11,3
1 2021,18,11,6,5
9 2021,32,11,19,5
10 2021,3,3,7,null
5 2021,24,12,5,5
2 2021,14,7,4,3


##Broj novo obolelih slucajeva COVID-19 po mesecima u svetu

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
corona_by_year = spark.sql("SELECT SUM(CAST(new_cases AS int)) AS new_cases,date_format(time, 'M Y') AS DATE from new_corona_set WHERE EXTRACT(YEAR FROM time)< 2022  GROUP BY location,date_format(time, 'M Y') HAVING location='World' ORDER BY new_cases") 
#corona_by_year.show(20)
display(corona_by_year)

new_cases,DATE
9370,1 2020
76100,2 2020
790307,3 2020
2419533,4 2020
2901366,5 2020
4292565,6 2020
7119061,7 2020
7942897,8 2020
8492498,9 2020
8557110,12 2022


## Ukupan broj novozarazenih od pocetka pandemije COVID-19 po drzavama

In [0]:
corona_by_country = spark.sql("SELECT SUM(new_cases) as new_cases,iso_code from new_corona_set WHERE location <> 'World' and  location <> 'High Income'  GROUP BY iso_code ORDER BY new_cases DESC") 
display(corona_by_country)

new_cases,iso_code
2.15577048E8,OWID_HIC
1.42223668E8,OWID_EUR
1.08583344E8,OWID_UMC
1.06156757E8,OWID_ASI
9.8737153E7,OWID_EUN
9.0789027E7,OWID_NAM
7.8600975E7,OWID_LMC
7.7428573E7,USA
5.1293498E7,OWID_SAM
4.2536137E7,IND


## H1 Video igre se uglavnom objavljuju za vise platformi

In [0]:
games = sqlContext.sql("select * from `df` ")
game_platform = games.groupBy("name").pivot("platform",["PC", "PlayStation 2", "PlayStation 3", "PlayStation 4", "PlayStation 5", "Xbox 360",  "Xbox One"]).count()

display(game_platform)

name,PC,PlayStation 2,PlayStation 3,PlayStation 4,PlayStation 5,Xbox 360,Xbox One
Double Kick Heroes,null,null,null,null,null,null,1
Joust,null,null,null,null,null,1,null
Human: Fall Flat,1,null,null,1,null,null,1
Dungeons 2,1,null,null,1,null,null,null
Volume,1,null,null,1,null,null,null
NCAA College Football 2K3,null,1,null,null,null,null,null
Anodyne,1,null,null,null,null,null,null
Back to the Future: The Game,null,null,null,null,null,null,null
Nioh: Dragon of the North,null,null,null,1,null,null,null
Masquerada: Songs and Shadows,null,null,null,1,null,null,null


## H2 Broj rasta zarazenih virusom COVID-19 je uzrokovao vecem broju objavljivanja video igara

In [0]:
covid_games_count = corona_by_year.join(date_game_count,corona_by_year["DATE"] == date_game_count["date"],"left_outer")
display(covid_games_count)

new_cases,DATE,date,count
9370,1 2020,1 2020,67
76100,2 2020,2 2020,78
790307,3 2020,3 2020,86
2419533,4 2020,4 2020,73
2901366,5 2020,5 2020,87
4292565,6 2020,6 2020,79
7119061,7 2020,7 2020,88
7942897,8 2020,8 2020,110
8492498,9 2020,9 2020,97
8557110,12 2022,null,null


## H3 Isti broj igara postoji za obe PS konzole

In [0]:
ps5_games = spark.sql("SELECT COUNT(name) FILTER( WHERE platform='PlayStation 5' ) AS PS5 , COUNT(name) FILTER( WHERE platform='PlayStation 4' ) AS PS4 FROM df WHERE EXTRACT(YEAR FROM release_date) >=2021 ") 
display(ps5_games)

PS5,PS4
93,93


## H4 Najbolji prosecni metascore ima igrica The Legend of Zelda: Ocarina of Time

In [0]:
meta_score_df = spark.sql("SELECT name, avg(meta_score) as meta_score FROM df GROUP BY name ORDER BY avg(meta_score) desc ")
display(meta_score_df)


name,meta_score
The Legend of Zelda: Ocarina of Time,99.0
Super Mario Galaxy,97.0
Super Mario Galaxy 2,97.0
The House in Fata Morgana - Dreams of the Revenants Edition -,97.0
NFL 2K1,97.0
Super Mario Odyssey,97.0
Metroid Prime,97.0
Grand Theft Auto V,96.75
The Legend of Zelda: Breath of the Wild,96.5
Gran Turismo,96.0


## H5 Serijal GTA postoji za cak 11 razlicitih platformi

In [0]:
games.filter(col("name").contains("Grand Theft Auto")).select("platform").distinct().show()

+----------------+
|        platform|
+----------------+
|Game Boy Advance|
|   PlayStation 2|
|     PlayStation|
|              PC|
|        Xbox One|
|   PlayStation 3|
|             PSP|
|            Xbox|
|   PlayStation 4|
|              DS|
|        Xbox 360|
+----------------+



## H6 Iskustvo korisnika na konzoli PS u vecem broju slucajeva bolje ocenjeno u odnosu na PC

In [0]:
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("name")
max_score_df = games.withColumn("max", max(col("user_review")).over(windowSpec)).select("name","max","platform","user_review").orderBy(desc("max"))


max_score_df.show(100)


+--------------------+----+-------------+-----------+
|                name| max|     platform|user_review|
+--------------------+----+-------------+-----------+
|Call of Duty 4: M...|94.0|           DS|        6.2|
|Call of Duty 4: M...|94.0|           PC|       92.0|
|Call of Duty 4: M...|94.0|PlayStation 3|       94.0|
|              Halo 3|94.0|     Xbox 360|       94.0|
|Call of Duty 4: M...|94.0|     Xbox 360|       94.0|
|      System Shock 2|92.0|           PC|       92.0|
|Uncharted 3: Drak...|92.0|PlayStation 3|       92.0|
|Age of Empires II...|92.0|           PC|       92.0|
|Dragon Quest XI S...|91.0|     Xbox One|        8.4|
|Dragon Quest XI S...|91.0|       Switch|       91.0|
|Dragon Quest XI S...|91.0|PlayStation 4|        8.1|
|Klonoa 2: Lunatea...|91.0|PlayStation 2|       91.0|
| Portal: Still Alive|90.0|     Xbox 360|       90.0|
|          NBA Street|89.0|PlayStation 2|       89.0|
|          NBA Street|89.0|     GameCube|        6.0|
| Fight Night Round 2|88.0|P

In [0]:
dbutils.fs.rm('/FileStore/tables/data/covid-serbia.csv', True)

Out[193]: False

## DATA STREAMING

In [0]:
from pyspark.sql.functions import *
from time import sleep

file_path = "/FileStore/tables/data-stream"

staticInputDF = (spark.read.option("header",True).csv(file_path))
 
schema = staticInputDF.schema

streamingInputDF = spark.readStream.schema(schema).option("header",True).option("maxFilesPerTrigger", 1).csv(file_path)


streamingCountsDF = streamingInputDF.filter(col("new_cases")>1000).groupBy(col("location")).count()


In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  

query = streamingCountsDF.writeStream.format("memory").queryName("counts").outputMode("complete").start()

In [0]:
%sql
select * from counts order by count desc

location,count
Europe,2021


In [0]:
%sql
select * from counts order by count desc

location,count
Europe,2691


In [0]:
query.stop()